In [1]:
import pandapower as pp
import pandapower.networks as nw
from copy import deepcopy
import pandapower.plotting as plot
import numpy as np
import pandas as pd
import os
import numba

import julia
julia.install()

from julia.api import Julia
jl = Julia(compiled_modules=False)

import warnings
warnings.filterwarnings('ignore')


[ Info: Julia version info


Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
      "Manjaro Linux"
  uname: Linux 6.6.26-1-MANJARO #1 SMP PREEMPT_DYNAMIC Wed Apr 10 20:11:08 UTC 2024 x86_64 unknown
  CPU: 13th Gen Intel(R) Core(TM) i5-13400F: 
                 speed         user         nice          sys         idle          irq
       #1-16  4605 MHz    3084759 s         27 s    4451801 s   36485344 s      21092 s
       
  Memory: 62.63359069824219 GB (30917.83984375 MB free)
  Uptime: 275832.0 sec
  Load Avg:  1.43  1.47  1.09
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, goldmont)
Environment:
  HOME = /home/iboero
  PATH = /home/iboero/miniconda3/envs/proy/bin:/home/iboero/.vscode-server/cli/servers/Stable-e170252f762678dec6ca2cc69aba1570769a5d39/server/bin/remote-cli:/home/iboero/miniconda3/envs/proy/bin:/home/iboero/miniconda3/condabin:/home/iboero/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/opt/cuda/bin:/opt/cuda/n

[ Info: Julia executable: /home/iboero/julia/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
│     libpython: /home/iboero/miniconda3/envs/proy/lib/libpython3.11.so.1.0
│ Python:
│     python: /home/iboero/miniconda3/envs/proy/bin/python
└     libpython: 


In [51]:
# Levantar red

red = "118" # o "118"


if red == "30":
  net = pp.networks.case30()
  net.line["max_i_ka"] *= 1.1
else:
  net = pp.networks.case118()

# Permitir solo a los generadores cambiar su potencia al resolver opf
net.load['controllable'] = False
net.gen['controllable'] = True
pp.runpm_ac_opf(net)

gen_p_nom = deepcopy(net.res_gen.p_mw.values)
load_p_nom = deepcopy(net.load['p_mw'])
load_q_nom = deepcopy(net.load['q_mvar'])

net.line["pm_param/target_branch"] = True  


In [52]:

# Empezar a generar datos

X = []
Y = []
X_LOAD = []
# Dividir en 10 batches de 1000 datos
for j in range(1):
  for i in range(100):
      y = np.zeros((len(net.bus["name"]),1))
      X_i = np.zeros((len(net.bus["name"]),3))
      print('iteracion:', i)
      # Tomar uniforme al rededor del valor nominal
      uniforme_activa_load = np.random.uniform(0.7,1.3,size=len(load_p_nom))
      uniforme_react_load = np.random.uniform(0.7,1.3,size=len(load_q_nom))
      uniforme_activa_gen = np.random.uniform(0.7,1.3,size=len(gen_p_nom))

      net.load.loc[:,'p_mw'] = uniforme_activa_load*load_p_nom
      net.load.loc[:,'q_mvar'] = uniforme_react_load*load_q_nom
      net.gen.loc[:,'p_mw'] =  uniforme_activa_gen*gen_p_nom
      net.gen.loc[:,"max_p_mw"] = net.gen['p_mw']
      net.gen.loc[:,"min_p_mw"] = net.gen['p_mw']

      # Resolver Flujo de carga
      pp.runpp(net,numba=False)
      X_i[net.gen["bus"]] += np.array([np.zeros(len(net.res_gen.p_mw)), np.zeros(len(net.res_gen.p_mw)),net.res_gen.p_mw]).T
      X_i[net.load["bus"]] += np.array([net.res_load.p_mw, net.res_load.q_mvar, np.zeros(len(net.res_load.q_mvar,))]).T
      # Resolver OPF
      try:
        pp.runpm_ploss(net,pm_time_limits=3)
        y[net.gen["bus"]] = net.res_gen.vm_pu.values.reshape(-1,1)
        Y.append(y)
        X.append(X_i)
      except:
        print("no convergio")
  X = np.array(X)
  Y = np.array(Y)
  # np.save('./data_reactiva/input'+red+f'/dato_{j}.npy',X)
  # np.save('./data_reactiva/vpm'+red+f'/dato_{j}.npy',Y)





iteracion: 0
iteracion: 1
iteracion: 2
iteracion: 3
iteracion: 4
iteracion: 5
iteracion: 6
iteracion: 7
iteracion: 8
iteracion: 9
iteracion: 10
iteracion: 11
iteracion: 12
iteracion: 13
iteracion: 14
iteracion: 15
iteracion: 16
iteracion: 17
iteracion: 18
iteracion: 19
iteracion: 20
iteracion: 21
iteracion: 22
iteracion: 23
iteracion: 24
iteracion: 25
iteracion: 26
iteracion: 27
iteracion: 28
iteracion: 29
iteracion: 30
iteracion: 31
iteracion: 32
iteracion: 33
iteracion: 34
iteracion: 35
iteracion: 36
iteracion: 37
iteracion: 38
iteracion: 39


OPF did not converge!


no convergio
iteracion: 40


OPF did not converge!


no convergio
iteracion: 41
iteracion: 42
iteracion: 43
iteracion: 44
iteracion: 45
iteracion: 46
iteracion: 47
iteracion: 48
iteracion: 49
iteracion: 50
iteracion: 51
iteracion: 52
iteracion: 53
iteracion: 54
iteracion: 55
iteracion: 56
iteracion: 57


...

no convergio
iteracion: 91
iteracion: 92


OPF did not converge!


no convergio
iteracion: 93
iteracion: 94
iteracion: 95


OPF did not converge!


no convergio
iteracion: 96
iteracion: 97
iteracion: 98
iteracion: 99


In [35]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,None,1,0.253900,0.111547,0.0,0.0,NaN,1.0,True,None,False
1,None,2,0.679685,0.005637,0.0,0.0,NaN,1.0,True,None,False
2,None,3,0.428650,0.000027,0.0,0.0,NaN,1.0,True,None,False
3,None,6,1.332202,0.172812,0.0,0.0,NaN,1.0,True,None,False
4,None,7,8.187207,0.055280,0.0,0.0,NaN,1.0,True,None,False
5,None,9,0.253170,0.032032,0.0,0.0,NaN,1.0,True,None,False
6,None,11,0.039173,0.001150,0.0,0.0,NaN,1.0,True,None,False
7,None,13,46.498310,0.057702,0.0,0.0,NaN,1.0,True,None,False
8,None,14,0.081444,4.253473,0.0,0.0,NaN,1.0,True,None,False
9,None,15,0.403559,0.005735,0.0,0.0,NaN,1.0,True,None,False


In [88]:
net.res_line.loading_percent

0     1.271364e+01
1     1.253398e+01
2     2.474605e+01
3     1.155810e+01
4     1.020948e+01
5     2.853213e+01
6     1.912956e+01
7     1.921300e+01
8     9.929674e+00
9     7.338228e+01
10    1.552382e+01
11    1.801872e+01
12    5.348547e-28
13    1.552382e+01
14    2.797843e+01
15    5.287459e+01
16    1.414969e+01
17    3.362744e+01
18    1.906787e+01
19    6.558538e+00
20    2.594862e+01
21    2.972080e+01
22    1.874364e+01
23    6.554478e+00
24    2.145326e+01
25    1.818566e+01
26    2.757883e+01
27    2.156738e+01
28    6.889115e+01
29    5.577978e+01
30    3.306734e+01
31    2.635071e+01
32    5.039352e+01
33    1.294495e+01
34    6.162666e+01
35    2.077654e+01
36    2.183370e+01
37    2.573730e+01
38    1.488080e+01
39    2.357172e+01
40    2.469272e+01
Name: loading_percent, dtype: float64

In [60]:
# Levantar los datos


X = np.load('./data_reactiva/PlQlPg'+red+f'.npy')
Y = np.load('./data_reactiva/vpu_opt'+red+f'.npy')

In [61]:
print(X.shape)
print(Y.shape)

(10, 30, 3)
(10, 30, 1)


In [70]:
net = pp.networks.case30()
net.line["max_i_ka"] *= 1.1

net.load.loc[:,'p_mw'] = X[0,net.load["bus"] , 0]
net.load.loc[:,'q_mvar'] = X[0 ,net.load["bus"] , 1]
net.gen.loc[:,'p_mw'] =   X[0 ,net.gen["bus"] , 2]
net.gen.loc[:,"max_p_mw"] = net.gen['p_mw']
net.gen.loc[:,"min_p_mw"] = net.gen['p_mw']
net.line["pm_param/target_branch"] = True  

net_loss = deepcopy(net)
pp.runpm_ploss(net_loss)

net.gen.loc[:,'vm_pu'] =   Y[0 ,net.gen["bus"]]
pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

